All instructions are provided for R. I am going to reproduce them in Python as best as I can.

# Preface

From the textbook, p. 199-200:
> In Sections 5.3.2 and 5.3.3, we saw that the `cv.glm()` function can be
used in order to compute the LOOCV test error estimate. Alternatively, one could compute those quantities using just the `glm()` and
`predict.glm()` functions, and a for loop. You will now take this approach in order to compute the LOOCV error for a simple logistic
regression model on the `Weekly` data set. Recall that in the context
of classification problems, the LOOCV error is given in (5.4).

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, \
                                          QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import statsmodels.api as sm


%matplotlib inline
sns.set()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
np.random.seed(1)
weekly = pd.read_csv('https://raw.githubusercontent.com/dsnair/ISLR/master/data/csv/Weekly.csv')
weekly = weekly.replace({'Up' : 1, 'Down' : 0})
x = weekly.drop('Direction', axis='columns')
y = weekly.Direction
weekly.head(3)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,0
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,0
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,1


# (a)

From the textbook, p. 200:
> Fit a logistic regression model that predicts `Direction` using `Lag1` and `Lag2`.

In [3]:
model_a = sm.GLM(y
                 , x[['Lag1', 'Lag2']]
                 , family=sm.genmod.families.family.Binomial()).fit()
print(model_a.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Direction   No. Observations:                 1089
Model:                            GLM   Df Residuals:                     1087
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -750.63
Date:                Mon, 19 Apr 2021   Deviance:                       1501.3
Time:                        21:17:54   Pearson chi2:                 1.09e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Lag1          -0.0322      0.026     -1.235      0.2

# (b)

From the textbook, p. 200:
> Fit a logistic regression model that predicts `Direction` using `Lag1` and `Lag2` using all but the first observation.

In [4]:
model_b = sm.GLM(y[1:]
                 , x.loc[1:, ['Lag1', 'Lag2']]
                 , family=sm.genmod.families.family.Binomial()).fit()
print(model_b.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Direction   No. Observations:                 1088
Model:                            GLM   Df Residuals:                     1086
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -749.89
Date:                Mon, 19 Apr 2021   Deviance:                       1499.8
Time:                        21:17:54   Pearson chi2:                 1.09e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Lag1          -0.0318      0.026     -1.223      0.2

# (c)

From the textbook, p. 200:
> Use the model from (b) to predict the direction of the first observation. You can do this by predicting that the first observation will go up if $P(Direction=Up | Lag1, \; Lag2) > 0.5$. Was this observation correctly classified?

In [5]:
'Up' if model_b.predict(x.loc[0:0, ['Lag1', 'Lag2']])[0] > 0.5 else 'Down'

'Up'

It was not correctly classified.

# (d)

From the textbook, p. 200:
> Write a for loop `from i = 1 to i = n`, where `n` is the number of observations in the data set, that performs each of the following steps:
1. Fit a logistic regression model using all but the ith observation to predict `Direction` using `Lag1` and `Lag2`.
1. Compute the posterior probability of the market moving up for the ith observation.
1. Use the posterior probability for the ith observation in order to predict whether or not the market moves up.
1. Determine whether or not an error was made in predicting the direction for the ith observation. If an error was made, then indicate this as a 1, and otherwise indicate it as a 0.

In [6]:
is_incorrect = []
for i, row in x.iterrows():
  model_d = sm.GLM(y.drop(i)
                   , x.drop(i)[['Lag1', 'Lag2']]
                   , family=sm.genmod.families.family.Binomial()).fit()
  pred = 1 if (model_b.predict(x.loc[i:i, ['Lag1', 'Lag2']]) > 0.5).all() else 1
  is_incorrect.append(0 if pred == y[i] else 1)

# (e)

From the textbook, p. 200:
> Take the average of the $n$ numbers obtained in (d).iv in order to obtain the LOOCV estimate for the test error. Comment on the results.

In [7]:
sum(is_incorrect) / len(is_incorrect)

0.4444444444444444

The LOOCV estimate of the test error is 44%. If you believe this estimate, the classifier does its job poorly.